## Sea Ice Diagnostics for two CESM3 runs

In [ ]:
import xarray as xr
import numpy as np
import yaml
import pandas as pd
import matplotlib.pyplot as plt
from dask.distributed import Client, LocalCluster

In [ ]:
CESM_output_dir = "/glade/campaign/cesm/development/cross-wg/diagnostic_framework/CESM_output_for_testing"
serial = False  # use dask LocalCluster
cases = [
    "g.e23_a16g.GJRAv4.TL319_t232_zstar_N65.2024.004",
    "g.e23_a16g.GJRAv4.TL319_t232_hycom1_N75.2024.005",
]

lc_kwargs = {}

begyr1 = 245
endyr1 = 305
begyr2 = 245
endyr2 = 305
nyears = 25

In [ ]:
# Spin up cluster (if running in parallel)
client = None
if not serial:
    cluster = LocalCluster(**lc_kwargs)
    client = Client(cluster)

client

In [ ]:
# Read in two cases. The ADF timeseries are needed here.

case1 = cases[0]
case2 = cases[1]

cbegyr1 = f"{begyr1:04d}"
cendyr1 = f"{endyr1:04d}"
cbegyr2 = f"{begyr2:04d}"
cendyr2 = f"{endyr2:04d}"

ds1 = xr.open_mfdataset(
    CESM_output_dir
    + "/"
    + case1
    + "/ts/"
    + case1
    + ".cice.h."
    + "*."
    + cbegyr1
    + "01-"
    + cendyr1
    + "12.nc",
    data_vars="minimal",
    compat="override",
    coords="minimal",
)
ds2 = xr.open_mfdataset(
    CESM_output_dir
    + "/"
    + case2
    + "/ts/"
    + case2
    + ".cice.h."
    + "*."
    + cbegyr2
    + "01-"
    + cendyr2
    + "12.nc",
    data_vars="minimal",
    compat="override",
    coords="minimal",
)

TLAT = ds1["TLAT"]
TLON = ds1["TLON"]
tarea = ds1["tarea"]

# Make a DataArray with the number of days in each month, size = len(time)
month_length = ds1.time.dt.days_in_month
weights_monthly = (
    month_length.groupby("time.year") / month_length.groupby("time.year").sum()
)


# seasons = xr.full_like(months, fill_value="none", dtype="U4")
# seasons.name = "season"
# seasons[months.isin([1, 2, 3])] = "JFM"
# seasons[months.isin([4, 5, 6])] = "AMJ"
# seasons[months.isin([7, 8, 9])] = "JAS"
# seasons[months.isin([10, 11, 12])] = "OND"
# weights_season = month_length.groupby(seasons) / month_length.groupby(seasons).sum()

ds1_ann = (ds1 * weights_monthly).resample(time="YS").sum(dim="time")
ds2_ann = (ds2 * weights_monthly).resample(time="YS").sum(dim="time")


# ds1_seas = (ds1 * weights_season).resample(time="QS-JAN").sum(dim="time")
# ds2_seas = (ds2 * weights_season).resample(time="QS-JAN").sum(dim="time")

with open("cice_masks.yml", "r") as file:
    cice_masks = yaml.safe_load(file)

with open("cice_vars.yml", "r") as file:
    cice_vars = yaml.safe_load(file)

print(ds1["aice"])

In [ ]:
from plot_diff import plot_diff
from vect_diff import vect_diff

In [ ]:
for var in cice_vars:
    vmin = cice_vars[var][0]["levels"][0]
    vmax = cice_vars[var][0]["levels"][-1]
    levels = np.array(cice_vars[var][0]["levels"])
    title = cice_vars[var][1]["title"]
    field1 = ds1_ann[var].isel(time=slice(-nyears, None)).mean("time").squeeze()
    field2 = ds2_ann[var].isel(time=slice(-nyears, None)).mean("time").squeeze()
    plot_diff(field1, field2, levels, case1, case2, title, "N", TLAT, TLON)

In [ ]:
for var in cice_vars:
    vmin = cice_vars[var][0]["levels"][0]
    vmax = cice_vars[var][0]["levels"][1]
    levels = np.array(cice_vars[var][0]["levels"])
    title = cice_vars[var][1]["title"]
    field1 = ds1_ann[var].isel(time=slice(-nyears, None)).mean("time").squeeze()
    field2 = ds2_ann[var].isel(time=slice(-nyears, None)).mean("time").squeeze()
    plot_diff(field1, field2, levels, case1, case2, title, "S", TLAT, TLON)

In [ ]:
ds1_area = (tarea * ds1.aice).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-12
ds2_area = (tarea * ds2.aice).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-12

ds1_vhi = (tarea * ds1.hi).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-13
ds2_vhi = (tarea * ds2.hi).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-13

ds1_vhs = (tarea * ds1.hs).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-13
ds2_vhs = (tarea * ds2.hs).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-13

fig = plt.figure(figsize=(10, 10), tight_layout=True)

ax = fig.add_subplot(3, 1, 1)
ds1_vhi.plot()
ds2_vhi.plot()

plt.ylim((0, 10))
plt.xlabel("Month")
plt.ylabel("NH Sea Ice Volume $m x 10^{13}$")
plt.legend([case1, case2])

ax = fig.add_subplot(3, 1, 2)
ds1_vhs.plot()
ds2_vhs.plot()

plt.ylim((0, 1))
plt.xlabel("Month")
plt.ylabel("NH Snow Volume $m x 10^{13}$")
plt.legend([case1, case2])

ax = fig.add_subplot(3, 1, 3)
ds1_area.plot()
ds2_area.plot()

plt.ylim((0, 25))
plt.xlabel("Month")
plt.ylabel("NH Sea Ice Area $m x 10^{12}$")
plt.legend([case1, case2])

In [ ]:
ds1_area_ann = (tarea * ds1_ann["aice"]).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-12
ds2_area_ann = (tarea * ds2_ann["aice"]).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-12

ds1_vhi_ann = (tarea * ds1_ann["hi"]).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-13
ds2_vhi_ann = (tarea * ds2_ann["hi"]).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-13

ds1_vhs_ann = (tarea * ds1_ann["hs"]).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-13
ds2_vhs_ann = (tarea * ds2_ann["hs"]).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-13

fig = plt.figure(figsize=(10, 10), tight_layout=True)

ax = fig.add_subplot(3, 1, 1)
ds1_vhi_ann.plot()
ds2_vhi_ann.plot()

plt.ylim((0, 10))
plt.xlabel("Year")
plt.ylabel("NH Annual Mean Sea Ice Volume $m x 10^{13}$")
plt.legend([case1, case2])

ax = fig.add_subplot(3, 1, 2)
ds1_vhs_ann.plot()
ds2_vhs_ann.plot()

plt.ylim((0, 1))
plt.xlabel("Year")
plt.ylabel("NH Annual Mean Snow Volume $m x 10^{13}$")
plt.legend([case1, case2])

ax = fig.add_subplot(3, 1, 3)
ds1_area_ann.plot()
ds2_area_ann.plot()

plt.ylim((0, 25))
plt.xlabel("Year")
plt.ylabel("NH Annual Mean Sea Ice Area $m x 10^{12}$")
plt.legend([case1, case2])

ds1_area.sel(time=ds1_area.time.dt.month.isin([10])).plot()
ds2_area.sel(time=ds2_area.time.dt.month.isin([10])).plot()

plt.ylim((0,25))
plt.xlabel("Year")
plt.ylabel("NH September Sea Ice Area $m x 10^{12}$")
plt.legend([case1,case2])

In [ ]:
ds1_area = (tarea * ds1.aice).where(TLAT < 0).sum(dim=["nj", "ni"]) * 1.0e-12
ds2_area = (tarea * ds2.aice).where(TLAT < 0).sum(dim=["nj", "ni"]) * 1.0e-12

ds1_area.plot()
ds2_area.plot()

plt.ylim((0, 25))
plt.xlabel("Month")
plt.ylabel("SH Sea Ice Area $m x 10^{12}$")
plt.legend([case1, case2])

In [ ]:
ds1_area_ann = (tarea * ds1_ann.aice).where(TLAT < 0).sum(dim=["nj", "ni"]) * 1.0e-12
ds2_area_ann = (tarea * ds2_ann.aice).where(TLAT < 0).sum(dim=["nj", "ni"]) * 1.0e-12

ds1_area_ann.plot()
ds2_area_ann.plot()

plt.ylim((0, 25))
plt.xlabel("Year")
plt.ylabel("SH Annual Mean Sea Ice Area $m x 10^{12}$")
plt.legend([case1, case2])

In [ ]:
### Read in the NSIDC data from files

path_nsidc = "/glade/campaign/cesm/development/pcwg/ice/data/NSIDC_SeaIce_extent/"

jan_nsidc = pd.read_csv(path_nsidc + "N_01_extent_v3.0.csv", na_values=["-99.9"])
feb_nsidc = pd.read_csv(path_nsidc + "N_02_extent_v3.0.csv", na_values=["-99.9"])
mar_nsidc = pd.read_csv(path_nsidc + "N_03_extent_v3.0.csv", na_values=["-99.9"])
apr_nsidc = pd.read_csv(path_nsidc + "N_04_extent_v3.0.csv", na_values=["-99.9"])
may_nsidc = pd.read_csv(path_nsidc + "N_05_extent_v3.0.csv", na_values=["-99.9"])
jun_nsidc = pd.read_csv(path_nsidc + "N_06_extent_v3.0.csv", na_values=["-99.9"])
jul_nsidc = pd.read_csv(path_nsidc + "N_07_extent_v3.0.csv", na_values=["-99.9"])
aug_nsidc = pd.read_csv(path_nsidc + "N_08_extent_v3.0.csv", na_values=["-99.9"])
sep_nsidc = pd.read_csv(path_nsidc + "N_09_extent_v3.0.csv", na_values=["-99.9"])
oct_nsidc = pd.read_csv(path_nsidc + "N_10_extent_v3.0.csv", na_values=["-99.9"])
nov_nsidc = pd.read_csv(path_nsidc + "N_11_extent_v3.0.csv", na_values=["-99.9"])
dec_nsidc = pd.read_csv(path_nsidc + "N_12_extent_v3.0.csv", na_values=["-99.9"])

jan_area = jan_nsidc.iloc[:, 5].values
feb_area = feb_nsidc.iloc[:, 5].values
mar_area = mar_nsidc.iloc[:, 5].values
apr_area = apr_nsidc.iloc[:, 5].values
may_area = may_nsidc.iloc[:, 5].values
jun_area = jun_nsidc.iloc[:, 5].values
jul_area = jul_nsidc.iloc[:, 5].values
aug_area = aug_nsidc.iloc[:, 5].values
sep_area = sep_nsidc.iloc[:, 5].values
oct_area = oct_nsidc.iloc[:, 5].values
nov_area = nov_nsidc.iloc[:, 5].values
dec_area = dec_nsidc.iloc[:, 5].values

jan_ext = jan_nsidc.iloc[:, 4].values
feb_ext = feb_nsidc.iloc[:, 4].values
mar_ext = mar_nsidc.iloc[:, 4].values
apr_ext = apr_nsidc.iloc[:, 4].values
may_ext = may_nsidc.iloc[:, 4].values
jun_ext = jun_nsidc.iloc[:, 4].values
jul_ext = jul_nsidc.iloc[:, 4].values
aug_ext = aug_nsidc.iloc[:, 4].values
sep_ext = sep_nsidc.iloc[:, 4].values
oct_ext = oct_nsidc.iloc[:, 4].values
nov_ext = nov_nsidc.iloc[:, 4].values
dec_ext = dec_nsidc.iloc[:, 4].values

print(dec_ext)
nsidc_clim = [
    np.nanmean(jan_ext[0:35]),
    np.nanmean(feb_ext[0:35]),
    np.nanmean(mar_ext[0:35]),
    np.nanmean(apr_ext[0:35]),
    np.nanmean(may_ext[0:35]),
    np.nanmean(jun_ext[0:35]),
    np.nanmean(jul_ext[0:35]),
    np.nanmean(aug_ext[0:35]),
    np.nanmean(sep_ext[0:35]),
    np.nanmean(oct_ext[0:35]),
    np.nanmean(nov_ext[0:35]),
    np.nanmean(dec_ext[0:35]),
]

plt.plot(nsidc_clim)

In [ ]:
aice1_month = ds1["aice"].groupby("time.month").mean(dim="time", skipna=True)
aice2_month = ds2["aice"].groupby("time.month").mean(dim="time", skipna=True)

mask_tmp1 = np.where(np.logical_and(aice1_month > 0.15, ds1["TLAT"] > 0), 1.0, 0.0)
mask_tmp2 = np.where(np.logical_and(aice2_month > 0.15, ds1["TLAT"] > 0), 1.0, 0.0)

mask_ext1 = xr.DataArray(data=mask_tmp1, dims=["month", "nj", "ni"])
mask_ext2 = xr.DataArray(data=mask_tmp2, dims=["month", "nj", "ni"])


ext1 = (mask_ext1 * tarea).sum(["ni", "nj"]) * 1.0e-12
ext2 = (mask_ext2 * tarea).sum(["ni", "nj"]) * 1.0e-12

plt.plot(ext1)
plt.plot(ext2)
plt.plot(nsidc_clim)

plt.ylim((0, 25))
plt.xlabel("Month")
plt.ylabel("Climatological Seasonal Cycle Ice Extent $m x 10^{12}$")
plt.legend([case1, case2, "NSIDC"])

In [ ]:
ds1_area = (tarea * ds1.aice).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-12
ds2_area = (tarea * ds2.aice).where(TLAT > 0).sum(dim=["nj", "ni"]) * 1.0e-12

ds1_sep = ds1_area.sel(time=(ds1_area.time.dt.month == 9))
ds2_sep = ds2_area.sel(time=(ds2_area.time.dt.month == 9))

plt.plot(ds1_sep)
plt.plot(ds2_sep)
plt.plot(sep_area)

plt.ylim((0, 25))
plt.xlabel("Year")
plt.ylabel("Sea Ice Area $mx10^{12}$")
plt.legend([case1, case2, "NSIDC"])

In [ ]:
latm = cice_masks["Lab_lat"]
lonm = cice_masks["Lab_lon"]

lon = np.where(TLON < 0, TLON + 360.0, TLON)

mask1 = np.where(np.logical_and(TLAT > latm[0], TLAT < latm[1]), 1.0, 0.0)
mask2 = np.where(np.logical_or(lon > lonm[0], lon < lonm[1]), 1.0, 0.0)
mask = mask1 * mask2

ds1_lab = (mask * tarea * ds1.aice).sum(dim=["nj", "ni"]) * 1.0e-12
ds2_lab = (mask * tarea * ds2.aice).sum(dim=["nj", "ni"]) * 1.0e-12

ds1_lab.plot()
ds2_lab.plot()

plt.ylim((0, 10))
plt.xlabel("Month")
plt.ylabel("Labrador Sea Ice Area $m x 10^{12}$")
plt.legend([case1, case2])

In [ ]:
uvel1 = ds1_ann["uvel"].isel(time=slice(-nyears, None)).mean("time").squeeze()
vvel1 = ds1_ann["vvel"].isel(time=slice(-nyears, None)).mean("time").squeeze()
uvel2 = ds2_ann["uvel"].isel(time=slice(-nyears, None)).mean("time").squeeze()
vvel2 = ds2_ann["vvel"].isel(time=slice(-nyears, None)).mean("time").squeeze()
ds_angle = xr.open_dataset(
    "/glade/campaign/cesm/development/cross-wg/diagnostic_framework/angle_tx2_3v2.nc"
)
angle = ds_angle["ANGLE"]

vect_diff(uvel1, vvel1, uvel2, vvel2, angle, "N", case1, case2, TLAT, TLON)

In [ ]:
vect_diff(uvel1, vvel1, uvel2, vvel2, angle, "S", case1, case2, TLAT, TLON)